In [12]:
import pandas as pd

In [16]:
file=r'C:\Users\ARUNA\Deep Learning Edvancer\Download Data\Pokemon.csv'


In [17]:
data=pd.read_csv(file)

In [18]:
data.head()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [22]:
data['Legendary'].value_counts(normalize=True)

False    0.91875
True     0.08125
Name: Legendary, dtype: float64

In [23]:
data['Legendary'].dtype

dtype('bool')

In [24]:
## first approach with simple dummies and model performance

In [25]:

data.head()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [26]:
data.nunique()

Name          800
Type 1         18
Type 2         18
Total         200
HP             94
Attack        111
Defense       103
Sp. Atk       105
Sp. Def        92
Speed         108
Generation      6
Legendary       2
dtype: int64

In [27]:
drop_cols=['Name','Total']
data.drop(drop_cols,1,inplace=True)

In [28]:
data.isnull().sum()

Type 1          0
Type 2        386
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [29]:
data['Type 2'].fillna('missing',inplace=True)

In [30]:
data['Legendary']=data['Legendary'].astype(int)

In [53]:
data1=data.copy()
data1.head()

,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Grass,Poison,45,49,49,65,65,45,1,0
1,Grass,Poison,60,62,63,80,80,60,1,0
2,Grass,Poison,80,82,83,100,100,80,1,0
3,Grass,Poison,80,100,123,122,120,80,1,0
4,Fire,missing,39,52,43,60,50,65,1,0


In [54]:
cat_cols=['Type 1','Type 2']


In [55]:
for col in cat_cols:
    freqs=data1[col].value_counts()
    cats=freqs.index[freqs>20]
    for cat in cats:
        name=col+'_'+cat
        data1[name]=(data1[col]==cat).astype(int)
    del data1[col]

In [56]:
data1.head()

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Type 1_Water,Type 1_Normal,...,Type 1_Ice,Type 2_missing,Type 2_Flying,Type 2_Ground,Type 2_Poison,Type 2_Psychic,Type 2_Fighting,Type 2_Grass,Type 2_Fairy,Type 2_Steel
0,45,49,49,65,65,45,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,60,62,63,80,80,60,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,80,82,83,100,100,80,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,80,100,123,122,120,80,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,39,52,43,60,50,65,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [57]:
data1.shape

(800, 33)

In [58]:
y=data1['Legendary']
x=data1.drop('Legendary',1)

In [59]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
rf=RandomForestClassifier(n_estimators=100,n_jobs=-1,bootstrap=True)

In [61]:
from sklearn.model_selection import cross_val_score

In [62]:
scores=cross_val_score(rf,x,y,scoring='roc_auc',cv=5)

In [67]:
scores.mean()

0.9626373626373628

In [64]:
## with categorical embeddings for Type 1 and Type 2

In [65]:
dummy_data=pd.get_dummies(data['Type 1'],prefix='type1')
dummy_data.shape[1]

18

In [66]:
y_type1=dummy_data

In [68]:
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Input

In [69]:
embedding_dim=3

inputs=Input(shape=(dummy_data.shape[1],))

dense1=Dense(15,activation='relu')(inputs)

embedded_output=Dense(embedding_dim)(dense1)

outputs=Dense(dummy_data.shape[1],activation='softmax')(embedded_output)

model=Model(inputs=inputs,outputs=outputs)

embedder=Model(inputs=inputs,outputs=embedded_output)

In [70]:
model.compile(optimizer='adam',loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(dummy_data,y_type1,epochs=100,batch_size=100)

Train on 800 samples
Epoch 1/100
800/800 [==============================] - 5s 6ms/sample - loss: 2.8837 - accuracy: 0.0000e+00
Epoch 2/100
800/800 [==============================] - 0s 125us/sample - loss: 2.8644 - accuracy: 0.0150
Epoch 3/100
800/800 [==============================] - 0s 60us/sample - loss: 2.8452 - accuracy: 0.2412
Epoch 4/100
800/800 [==============================] - 0s 101us/sample - loss: 2.8264 - accuracy: 0.3500
Epoch 5/100
800/800 [==============================] - 0s 68us/sample - loss: 2.8075 - accuracy: 0.3500
Epoch 6/100
800/800 [==============================] - 0s 109us/sample - loss: 2.7887 - accuracy: 0.3500
Epoch 7/100
800/800 [==============================] - 0s 70us/sample - loss: 2.7685 - accuracy: 0.3500
Epoch 8/100
800/800 [==============================] - 0s 79us/sample - loss: 2.7476 - accuracy: 0.3500
Epoch 9/100
800/800 [==============================] - 0s 175us/sample - loss: 2.7260 - accuracy: 0.3500
Epoch 10/100
800/800 [==============

800/800 [==============================] - 0s 41us/sample - loss: 0.6219 - accuracy: 0.7812
Epoch 79/100
800/800 [==============================] - 0s 41us/sample - loss: 0.6051 - accuracy: 0.7812
Epoch 80/100
800/800 [==============================] - 0s 40us/sample - loss: 0.5891 - accuracy: 0.7812
Epoch 81/100
800/800 [==============================] - 0s 39us/sample - loss: 0.5734 - accuracy: 0.7962
Epoch 82/100
800/800 [==============================] - 0s 38us/sample - loss: 0.5577 - accuracy: 0.8163
Epoch 83/100
800/800 [==============================] - 0s 36us/sample - loss: 0.5420 - accuracy: 0.8163
Epoch 84/100
800/800 [==============================] - 0s 41us/sample - loss: 0.5269 - accuracy: 0.8163
Epoch 85/100
800/800 [==============================] - 0s 33us/sample - loss: 0.5118 - accuracy: 0.8188
Epoch 86/100
800/800 [==============================] - 0s 33us/sample - loss: 0.4968 - accuracy: 0.9125
Epoch 87/100
800/800 [==============================] - 0s 30us/samp

In [71]:
dummy_data.head()

,type1_Bug,type1_Dark,type1_Dragon,type1_Electric,type1_Fairy,type1_Fighting,type1_Fire,type1_Flying,type1_Ghost,type1_Grass,type1_Ground,type1_Ice,type1_Normal,type1_Poison,type1_Psychic,type1_Rock,type1_Steel,type1_Water
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [72]:
low_dim_type1=pd.DataFrame(embedder.predict(dummy_data),columns=['t1_emb1','t1_emb2','t1_emb3'])
low_dim_type1.head()

,t1_emb1,t1_emb2,t1_emb3
0,-2.851535,-0.303081,6.783140
1,-2.851535,-0.303081,6.783140
2,-2.851535,-0.303081,6.783140
3,-2.851535,-0.303081,6.783140
4,4.186382,-3.683265,0.449757


In [73]:
dummy_data=pd.get_dummies(data['Type 2'],prefix='type1')
dummy_data.shape[1]

19

In [120]:
y_type2=dummy_data


embedding_dim=3
inputs=Input(shape=(dummy_data.shape[1],))
dense1=Dense(15,activation='relu')(inputs)
embedded_output=Dense(embedding_dim)(dense1)
outputs=Dense(dummy_data.shape[1],activation='softmax')(embedded_output)
model=Model(inputs=inputs,outputs=outputs)

embedder=Model(inputs=inputs,outputs=embedded_output)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(dummy_data,y_type2,epochs=200,batch_size=100)

low_dim_type2=pd.DataFrame(embedder.predict(dummy_data),columns=['t2_emb1','t2_emb2','t2_emb3'])


Train on 800 samples
Epoch 1/200
800/800 [==============================] - 2s 2ms/sample - loss: 2.9544 - accuracy: 0.0000e+00
Epoch 2/200
800/800 [==============================] - 0s 103us/sample - loss: 2.9221 - accuracy: 0.0000e+00
Epoch 3/200
800/800 [==============================] - 0s 79us/sample - loss: 2.8916 - accuracy: 0.0375
Epoch 4/200
800/800 [==============================] - 0s 88us/sample - loss: 2.8625 - accuracy: 0.1213
Epoch 5/200
800/800 [==============================] - 0s 60us/sample - loss: 2.8351 - accuracy: 0.1213
Epoch 6/200
800/800 [==============================] - 0s 63us/sample - loss: 2.8074 - accuracy: 0.1213
Epoch 7/200
800/800 [==============================] - 0s 74us/sample - loss: 2.7786 - accuracy: 0.1213
Epoch 8/200
800/800 [==============================] - 0s 58us/sample - loss: 2.7496 - accuracy: 0.1213
Epoch 9/200
800/800 [==============================] - 0s 59us/sample - loss: 2.7183 - accuracy: 0.1213
Epoch 10/200
800/800 [=============

800/800 [==============================] - 0s 55us/sample - loss: 0.4325 - accuracy: 0.8525
Epoch 78/200
800/800 [==============================] - 0s 39us/sample - loss: 0.4252 - accuracy: 0.8525
Epoch 79/200
800/800 [==============================] - 0s 44us/sample - loss: 0.4183 - accuracy: 0.8525
Epoch 80/200
800/800 [==============================] - 0s 40us/sample - loss: 0.4115 - accuracy: 0.8525
Epoch 81/200
800/800 [==============================] - 0s 43us/sample - loss: 0.4047 - accuracy: 0.8525
Epoch 82/200
800/800 [==============================] - 0s 33us/sample - loss: 0.3982 - accuracy: 0.8525
Epoch 83/200
800/800 [==============================] - 0s 36us/sample - loss: 0.3918 - accuracy: 0.8650
Epoch 84/200
800/800 [==============================] - 0s 31us/sample - loss: 0.3854 - accuracy: 0.8963
Epoch 85/200
800/800 [==============================] - 0s 34us/sample - loss: 0.3789 - accuracy: 0.9062
Epoch 86/200
800/800 [==============================] - 0s 36us/samp

800/800 [==============================] - 0s 39us/sample - loss: 0.1134 - accuracy: 0.9663
Epoch 155/200
800/800 [==============================] - 0s 58us/sample - loss: 0.1112 - accuracy: 0.9663
Epoch 156/200
800/800 [==============================] - 0s 39us/sample - loss: 0.1091 - accuracy: 0.9675
Epoch 157/200
800/800 [==============================] - 0s 45us/sample - loss: 0.1068 - accuracy: 0.9837
Epoch 158/200
800/800 [==============================] - 0s 50us/sample - loss: 0.1047 - accuracy: 0.9912
Epoch 159/200
800/800 [==============================] - 0s 44us/sample - loss: 0.1026 - accuracy: 0.9912
Epoch 160/200
800/800 [==============================] - 0s 41us/sample - loss: 0.1004 - accuracy: 0.9912
Epoch 161/200
800/800 [==============================] - 0s 55us/sample - loss: 0.0982 - accuracy: 0.9912
Epoch 162/200
800/800 [==============================] - 0s 58us/sample - loss: 0.0960 - accuracy: 0.9912
Epoch 163/200
800/800 [==============================] - 0s 

In [130]:
data.columns

Index(['Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def',
       'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [136]:
final_df=pd.concat([low_dim_type1,low_dim_type2,data1.select_dtypes(['int64']),data['Legendary']],1)
final_df


,t1_emb1,t1_emb2,t1_emb3,t2_emb1,t2_emb2,t2_emb3,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,-2.851535,-0.303081,6.783140,1.518698,-12.859267,-3.005494,45,49,49,65,65,45,1,0
1,-2.851535,-0.303081,6.783140,1.518698,-12.859267,-3.005494,60,62,63,80,80,60,1,0
2,-2.851535,-0.303081,6.783140,1.518698,-12.859267,-3.005494,80,82,83,100,100,80,1,0
3,-2.851535,-0.303081,6.783140,1.518698,-12.859267,-3.005494,80,100,123,122,120,80,1,0
4,4.186382,-3.683265,0.449757,1.393294,-4.055191,6.572913,39,52,43,60,50,65,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,-4.384644,-0.640538,-1.762678,-5.485688,5.222552,-2.499127,50,100,150,100,150,50,6,1
796,-4.384644,-0.640538,-1.762678,-5.485688,5.222552,-2.499127,50,160,110,160,110,110,6,1
797,3.838170,2.172629,-3.533895,6.983332,-4.146227,0.330945,80,110,60,150,130,70,6,1
798,3.838170,2.172629,-3.533895,0.141206,8.698248,1.511372,80,160,60,170,130,80,6,1


In [137]:
x=final_df

In [138]:
rf=RandomForestClassifier(n_estimators=100,n_jobs=-1,bootstrap=True)

In [ ]:
scores=cross_val_score(rf,x,y,scoring='roc_auc',cv=5)

In [ ]:
scores